<a href="https://colab.research.google.com/github/hihi5456/pytorch/blob/main/Fundusdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install adabelief-pytorch
!pip install timm

     |████████████████████████████████| 296kB 11.6MB/s 


In [1]:
import torchvision
from torchvision import transforms, datasets

import torch.utils.data.dataloader
import matplotlib.pyplot as plt


import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,ConcatDataset
import adabelief_pytorch
import numpy  as np

from sklearn.metrics import roc_auc_score, auc


ModuleNotFoundError: ignored

In [ ]:
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD, IMAGENET_INCEPTION_MEAN, IMAGENET_INCEPTION_STD
from timm.models.efficientnet_blocks import round_channels, resolve_bn_args, resolve_act_layer, BN_EPS_TF_DEFAULT
from timm.models.efficientnet_builder import EfficientNetBuilder, decode_arch_def, efficientnet_init_weights
from timm.models.features import FeatureInfo, FeatureHooks
from timm.models.helpers import build_model_with_cfg, default_cfg_for_features
from timm.models.layers import create_conv2d, create_classifier
from timm.models.registry import register_model
import timm
import timm.models.efficientnet

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(456)
if device =='cuda':
    torch.cuda.manual_seed_all(456)

In [14]:
class skipblock(nn.Module):
  def __init__(self, in_channel,out_channel,outsize):
    super(skipblock, self).__init__()
    
    
    self.conv0=nn.Conv2d(in_channel,out_channel,1)
    self.silu=nn.SiLU()
    
    self.block=nn.Sequential(
                    
            nn.Conv2d(out_channel, 64*in_channel, 1),
            nn.BatchNorm2d(in_channel*64),         
            nn.SiLU(),
            nn.Dropout2d(),
            

            nn.Conv2d(in_channel*64, in_channel*64, 3, 1, 1),
            nn.BatchNorm2d(in_channel*64),          
            nn.SiLU(),
            nn.Dropout2d(),

            nn.Conv2d(in_channel*64,out_channel,1),
            nn.BatchNorm2d(out_channel),
            nn.SiLU(),
            nn.Dropout2d(),
            
        )
    
    self.bn=nn.BatchNorm2d(out_channel)
    self.pool=nn.AdaptiveAvgPool2d(outsize)

  def forward(self, x):

    out=self.conv0(x)
    out=self.silu(out)
    nn.Dropout2d()
    skip=out
    #print(out.shape)
    out=self.block(out)
    #print(out.shape)
    out=out+skip
    #print(out.shape)
    out=self.bn(out)
    out=self.pool(out)

    return out

In [15]:
#gelu=nn.GELU()
class testblock1(nn.Module):
    def __init__(self,ni,num_class):
        super(testblock1, self).__init__()
  
        self.block0=skipblock(ni,32,300)
        
        self.block1=skipblock(32,3,100)
        
        self.block2=skipblock(3,1,10)
        
     
        self.classifier = nn.Sequential(
            nn.Linear(1*10*10,num_class),
            
        )

    def forward(self,x):
        
        out=self.block0(x)
        out=self.block1(out)
        out=self.block2(out)


        out = out.view(out.size(0),-1)
        out=self.classifier(out)
        
        return out


In [ ]:
model=timm.models.efficientnet_l2(pretrained=False)
model.reset_classifier(39)
model.drop_rate=0.5

model=model.to(device)

In [16]:
model=testblock1(3,39).to(device)

In [ ]:
model=timm.models.efficientnet.efficientnet_b7(pretrained=False).to(device)
model.reset_classifier(39)
model.drop_rate=0.5
model=model.to(device)

In [17]:
a=torch.rand((1,3,600,600)).to(device)
model(a)

RuntimeError: ignored

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
PATH='/content/gdrive/MyDrive/Colab Notebooks/fundusdataset/1000images/'

In [ ]:
transform = transforms.Compose(
    [
     transforms.RandomRotation(45),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     
     transforms.Resize((600,600)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
     ]
     )


In [ ]:
fundus_data = torchvision.datasets.ImageFolder(root=PATH,transform=transform)


In [ ]:
train_size=int(0.8*len(fundus_data))
test_size=len(fundus_data)-train_size

In [ ]:
train_set, test_set=torch.utils.data.random_split(fundus_data,[train_size,test_size])

In [ ]:
torch.save(train_set,PATH+'trainset.csv')
torch.save(test_set,PATH+'testset.csv')

In [ ]:
train_set=torch.load(PATH+'trainset.csv')
test_set=torch.load(PATH+'testset.csv')

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set,drop_last=True,
                                          batch_size=8,
                                          shuffle=True,
                                          num_workers=2,
                                          )
test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size=1,
                                          shuffle=False,
                                          num_workers=4,
                                          )

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
#optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
optimizer=adabelief_pytorch.AdaBelief(model.parameters(),lr=0.001)
#lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)


Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief
Rectification enabled in AdaBelief


In [ ]:
trainedepochs=0
print(len(train_loader))

epochs = 500


for epoch in range(epochs):
  model.train()
  
  running_loss = 0.0

  for i, data in enumerate(train_loader, 0):
    # get the inputs
    inputs, labels = data
    inputs=inputs.to(device)
    labels = labels.to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(inputs)

    
    
       # (numarray.max()/numarray[labels[0][0].to(torch.int).tolist()-1])
    #loss = (1/numnum[labels[0][0].to(torch.int).tolist()-1])*criterion(outputs, labels.to(torch.float)/10)
    loss = criterion(outputs, labels)
    
    #print( (numnum.max()/numnum[labels[0][0].to(torch.int).tolist()-1]))
    loss.backward()
    optimizer.step()

        # print statistics
    running_loss += loss.item()
    if i % 20 == 19:    
      print('[%d, %5d] loss: %.3f' %
            (epoch + 1, i + 1, running_loss / 20))
      running_loss = 0.0
    #lr_sche.step()
  
  if epoch % 50==49:
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
      for data in test_loader:
        
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        #print(predicted ,labels)
        correct += (predicted == labels).sum().item()

      print('Accuracy of the network on the {} test images: {} %%'.format(len(test_set),
                100 * correct / total))

        


  #if ( epoch==100 or epoch==150 or epoch==200 or epoch==250 or epoch==300 or epoch==350 or epoch==400 or epoch==450 or epoch==500 or epoch==550):
      
  if (epoch==100 or epoch==200 or epoch==300 or  epoch==400 or epoch==500 ):
    torch.save(optimizer.state_dict, PATH+'optimizer{}.pt'.format(trainedepochs+epoch))
    torch.save(model.state_dict,PATH+'model{}.pt'.format(trainedepochs+epoch))

        
        
      
      # loop over the dataset multiple times

    #Check Accuracy
    #acc = acc_check(resnet50, testloader, epoch, save=1)
    
    
torch.save(model.state_dict,PATH+'model{}.pt'.format(trainedepochs+epochs))
torch.save(optimizer.state_dict, PATH+'optimizer{}.pt'.format(trainedepochs+epochs))
print('Finished Training')

100
[1,    20] loss: 3.737
[1,    40] loss: 3.700
[1,    60] loss: 3.698
[1,    80] loss: 3.737
[1,   100] loss: 3.691
[2,    20] loss: 3.635
[2,    40] loss: 3.717
[2,    60] loss: 3.650
[2,    80] loss: 3.538
[2,   100] loss: 3.592
[3,    20] loss: 3.516
[3,    40] loss: 3.543
[3,    60] loss: 3.449
[3,    80] loss: 3.447
[3,   100] loss: 3.441
[4,    20] loss: 3.395
[4,    40] loss: 3.435
[4,    60] loss: 3.348
[4,    80] loss: 3.433
[4,   100] loss: 3.446
[5,    20] loss: 3.311
[5,    40] loss: 3.333
[5,    60] loss: 3.407
[5,    80] loss: 3.291
[5,   100] loss: 3.301
[6,    20] loss: 3.195
[6,    40] loss: 3.273
[6,    60] loss: 3.095
[6,    80] loss: 3.323
[6,   100] loss: 3.350
[7,    20] loss: 3.122
[7,    40] loss: 3.248
[7,    60] loss: 3.135
[7,    80] loss: 3.323
[7,   100] loss: 3.168
[8,    20] loss: 3.181
[8,    40] loss: 3.234
[8,    60] loss: 3.277
[8,    80] loss: 3.058
[8,   100] loss: 3.044
[9,    20] loss: 3.116
[9,    40] loss: 3.083
[9,    60] loss: 3.037
[9,    

In [ ]:
correct = 0
total = 0
model.eval() 
with torch.no_grad():
    for data in test_loader:
        
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
         
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        #print(predicted ,labels)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the {} test images: {} %%'.format(len(test_set),
    100 * correct / total))